<a href="https://colab.research.google.com/github/dangcaptkd2/PracticeWithTensorFlow/blob/main/Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 3.3MB/s 


In [ ]:
import kerastuner as kt

# Download and prepare dataset

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# Define the model

Use a model builder function to define the image classification model

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

#Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - ```RandomSearch```, ```Hyperband```, ```BayesianOptimization```, and ```Sklearn```. 

We use Hyperband tuner in this tutorial

To instantiate the Hyperband tuner, we must specify the hypermodel, the objective to optimize and the maximum number of epochs to train (max_epochs).






In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Create a callback to stop training early after reaching a certain value for the validation loss.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for ```tf.keras.model.fit``` in addition to the callback above.




In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 37s]
val_accuracy: 0.8604999780654907

Best val_accuracy So Far: 0.890500009059906
Total elapsed time: 00h 10m 48s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 192 and the optimal learning rate for the optimizer
is 0.001.



# Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6483 - accuracy: 0.7698 - val_loss: 0.4199 - val_accuracy: 0.8487
Epoch 2/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3886 - accuracy: 0.8587 - val_loss: 0.3794 - val_accuracy: 0.8658
Epoch 3/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3388 - accuracy: 0.8753 - val_loss: 0.3761 - val_accuracy: 0.8662
Epoch 4/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3085 - accuracy: 0.8881 - val_loss: 0.3537 - val_accuracy: 0.8721
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3012 - accuracy: 0.8889 - val_loss: 0.3243 - val_accuracy: 0.8848
Epoch 6/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2727 - accuracy: 0.8987 - val_loss: 0.3270 - val_accuracy: 0.8816
Epoch 7/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2628 - accuracy: 0.9015 - val_loss: 0.3224 - val_accuracy:

In [ ]:
best_epoch

17

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_test, label_test, epochs=best_epoch)

Epoch 1/17
313/313 [==============================] - 1s 3ms/step - loss: 0.9384 - accuracy: 0.6766
Epoch 2/17
313/313 [==============================] - 1s 3ms/step - loss: 0.5134 - accuracy: 0.8192
Epoch 3/17
313/313 [==============================] - 1s 3ms/step - loss: 0.4577 - accuracy: 0.8391
Epoch 4/17
313/313 [==============================] - 1s 3ms/step - loss: 0.4095 - accuracy: 0.8482
Epoch 5/17
313/313 [==============================] - 1s 3ms/step - loss: 0.3960 - accuracy: 0.8522
Epoch 6/17
313/313 [==============================] - 1s 3ms/step - loss: 0.3570 - accuracy: 0.8716
Epoch 7/17
313/313 [==============================] - 1s 3ms/step - loss: 0.3334 - accuracy: 0.8763
Epoch 8/17
313/313 [==============================] - 1s 3ms/step - loss: 0.3131 - accuracy: 0.8855
Epoch 9/17
313/313 [==============================] - 1s 3ms/step - loss: 0.3126 - accuracy: 0.8853
Epoch 10/17
313/313 [==============================] - 1s 3ms/step - loss: 0.2968 - accuracy: 0.8934